In [1]:
import os

import streamlit as st
from dotenv import load_dotenv

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [3]:
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate

In [4]:
# load_dotenv(".env")
# api_key = os.environ.get("OPENAI_API_KEY")

In [5]:
token_size = 200
vector_store_persist_directory = f"./vector-store/asu_ai_db_tr_{token_size}_token"
original_document_path = "./data/yeni_sss.txt"

In [6]:
embedding_function = OpenAIEmbeddings()

In [7]:
if os.path.exists(vector_store_persist_directory):
    # Load documents
    loader = TextLoader(original_document_path, encoding="UTF-8")
    documents = loader.load()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=token_size)

    # Split and persist documents
    docs = text_splitter.split_documents(documents)
    db = Chroma.from_documents(
        docs, embedding_function, persist_directory=vector_store_persist_directory
    )
    db.persist()
else:
    db = Chroma(persist_directory=vector_store_persist_directory, embedding_function=embedding_function)

In [8]:
llm = ChatOpenAI(temperature=0)

In [9]:
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(),
    llm=llm,
)

In [10]:
def answer_my_question(question):
    unique_docs = retriever_from_llm.get_relevant_documents(query=question)

    human_message_template = "{question} sorusuna aşağıda verilen bağlamdaki bilgilere ugun şekilde cevap ver.\n{baglam}." 
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_message_template)
    system_message_template = "Cevapta sadece sorunun en kısa cevabını döndür. Soru yeterince açık değilse soru alternatifleri vererek tekrar sorulmasını iste."
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_message_template)

    prompt = ChatPromptTemplate.from_messages([human_message_prompt,system_message_prompt])
    formatted_question = prompt.format_prompt(question=question, baglam=unique_docs).to_messages()
    result = llm(formatted_question)
    return result.content

In [17]:
answer_my_question("tezsiz yüksek lisans programlara ücret ödeyecek miyim?")

'Evet, tezsiz yüksek lisans programları genellikle ücretli olarak sunulmaktadır.'

In [22]:
answer_my_question('Lisansüstü programlara ücret ödeyecek miyim? Doktora, tezsiz yüksek lisans ve yüksek lisans için ücretler kaç TL?')

'Lisansüstü programlara ücret ödeyip ödemeyeceğiniz üniversiteye ve programın türüne bağlıdır. Üniversiteler ve programlar arasında ücret politikaları farklılık gösterebilir. Bu nedenle, ücretler hakkında kesin bir bilgi almak için ilgili üniversitenin resmi web sitesini veya ilgili bölümü ile iletişime geçmeniz önerilir.'

In [24]:
answer_my_question('Lisansüstü programlara ücret ödeyecek miyim?')

'Bu sorunun yanıtı, hangi lisansüstü programı seçtiğinize ve hangi üniversitede okumayı planladığınıza bağlıdır. Lisansüstü programlar genellikle ücretli olabilir, ancak bazı üniversiteler burs veya mali destek imkanları sunabilir. Bu nedenle, ücret ödeyip ödemeyeceğinizi öğrenmek için seçtiğiniz programın ve üniversitenin web sitesini kontrol etmeniz önemlidir.'

In [20]:
questions = [
                "Lisansüstü programlara ücret ödeyecek miyim? Doktora, tezsiz yüksek lisans ve yüksek lisans için ücretler kaç TL?",
                "Lisansüstü programlara ücret ödeyecek miyim?",
                "hangi lisansüstü programlar ücretsizdir?",
                "hangi lisansüstü programlar ücretlidir?",
                "tezsiz yüksek lisans programlara ücret ödeyecek miyim?",
                "şehit ve gazi yakınları için ücretli mi?",
                "burs başvurusu yapabilir miyim?",
                "nasıl burs başvurusu yapabilirim?",
                "nasıl yatay geçiş yapabilirim",
                "Lisansüstü programlarda kaç ders alırım",
                "Tezsiz Yüksek Lisansta kaç ders alırım",
                "Tezli Yüksek Lisansta kaç ders alırım",
                "doktorada kaç ders alırım",
                "yüksek lisans tezi nasıl yazılır?",
                "yüksek lisans tez savunması nasıl yapılır?",
                "tez savunmasında başarısız öğrenciler ne yapmalı?",
                "tez nasıl yazılır?",    
            ]

In [21]:
for q in questions:
    print(f"answer_my_question('{q}')")

answer_my_question('Lisansüstü programlara ücret ödeyecek miyim? Doktora, tezsiz yüksek lisans ve yüksek lisans için ücretler kaç TL?')
answer_my_question('Lisansüstü programlara ücret ödeyecek miyim?')
answer_my_question('hangi lisansüstü programlar ücretsizdir?')
answer_my_question('hangi lisansüstü programlar ücretlidir?')
answer_my_question('tezsiz yüksek lisans programlara ücret ödeyecek miyim?')
answer_my_question('şehit ve gazi yakınları için ücretli mi?')
answer_my_question('burs başvurusu yapabilir miyim?')
answer_my_question('nasıl burs başvurusu yapabilirim?')
answer_my_question('nasıl yatay geçiş yapabilirim')
answer_my_question('Lisansüstü programlarda kaç ders alırım')
answer_my_question('Tezsiz Yüksek Lisansta kaç ders alırım')
answer_my_question('Tezli Yüksek Lisansta kaç ders alırım')
answer_my_question('doktorada kaç ders alırım')
answer_my_question('yüksek lisans tezi nasıl yazılır?')
answer_my_question('yüksek lisans tez savunması nasıl yapılır?')
answer_my_question

In [13]:
# qnas = []

In [14]:
# import time

# for question in questions:
#     qnas.append(
#         {
#             "question":question,
#              "answer": answer_my_question(question)
#         }
#     )
#     time.sleep(2)

In [15]:
# for item in qnas:
#     print(f"## Soru: {item['question']}")
#     print(f"Cevap: {item['answer']}")
#     print("")